# DEPRECATED

use the scripts add_duplex_filter_columns.py and filter_duplex_variants.py now

This notebook filters the output of duplex_caller to only true mutations. It also contains the code for determining the minimum number of supporting reads to be confident in mutation calls.

In [ ]:
import sys
import os
sys.path.append(os.getcwd() + '/../python_scripts') # this lets us import files in python_scripts (like gtools)
import gtools
if os.getcwd()[:8] != '/scratch': # switch to the scratch directory where all the data files are
    os.chdir(f'/scratch/cam02551/{os.getcwd().split("/")[-2]}')

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['svg.fonttype'] = 'none'
import pandas as pd
import csv
import subprocess
from tqdm import tqdm
import importlib
import numpy as np
from functools import reduce
from collections import defaultdict
import multiprocessing
import pysam
import math
import scipy.stats as stats



In [ ]:
# sample_names = 'Col-0-1 Col-0-2 Col-0-3 Col-0-4 Col-0-5 Col-0-6 Col-0-7 Col-0-8 ' \
               # 'Col-0-swapped-1 Col-0-swapped-2 Col-0-swapped-3 Col-0-swapped-4 Col-0-swapped-5 Col-0-swapped-6 ' \
               #  'Ler-0-germline ' \
               #  'UVB-1 UVB-2 UVC-002 UVC-005 UVC-01 UVC-02 ' \
                # 'UVCnotime-0h UVCnotime-1h UVCnotreat-C1 UVCnotreat-C2'
                # Col-0-bonus1 Col-0-bonus2 Col-0-bonus3\
                # arp-1 arp-2 \
                # atx12r7-1 atx12r7-2 atx12r7-3 \
                # chr8-1 chr8-2 chr8-3 \
                # clf-1 clf-2 clf-3 \
                # ddm1-1 ddm1-2 ddm1-3 \
                # h2aw7-1 h2aw7-2 h2aw7-3 \
                # h2ax35-1 h2ax35-2 h2ax35-3 \
                # ku80-1 ku80-2 ku80-3 \
                # met1-epiRIL12-1 met1-epiRIL12-2 met1-epiRIL12-3 \
                # msh2-1 msh2-2 msh2-3 \
                # msh6-1 msh6-2 msh6-3 \
                # nrpd1b-1 nrpd1b-2 nrpd1b-3 \
                # parp2-1 parp2-2 parp2-3 \
                # poly-1 poly-2 poly-3 \
                # rad5a-1 rad5a-2 rad5a-3 \
                # rad7a-1 rad7a-2 rad7a-3 \
                # sdg8-1 sdg8-2 sdg8-3 \
                # suvh456-2 suvh456-3 \
                # xpg-1 xpg-2 xpg-3'
# sample_names = sample_names.split()

# sample_names = 'Col-0-1 Col-0-2 Col-0-3 Col-0-4 Col-0-5 Col-0-6 Col-0-7 Col-0-8 \
#                 Col-0-1_merged_01 Col-0-1_merged_02 Col-0-1_merged_04 Col-0-1_merged_06 Col-0-1_merged_08 \
#                 Col-0-2_merged_01 Col-0-2_merged_02 Col-0-2_merged_04 Col-0-2_merged_06 Col-0-2_merged_08 \
#                 Col-0-3_merged_01 Col-0-3_merged_02 Col-0-3_merged_04 Col-0-3_merged_06 Col-0-3_merged_08 \
#                 Col-0-4_merged_01 Col-0-4_merged_02 Col-0-4_merged_04 Col-0-4_merged_06 Col-0-4_merged_08 \
#                 Col-0-5_merged_01 Col-0-5_merged_02 Col-0-5_merged_04 Col-0-5_merged_06 Col-0-5_merged_08 \
#                 Col-0-6_merged_01 Col-0-6_merged_02 Col-0-6_merged_04 Col-0-6_merged_06 Col-0-6_merged_08 \
#                 Col-0-7_merged_01 Col-0-7_merged_02 Col-0-7_merged_04 Col-0-7_merged_06 Col-0-7_merged_08 \
#                 Col-0-8_merged_01 Col-0-8_merged_02 Col-0-8_merged_04 Col-0-8_merged_06 Col-0-8_merged_08 \
#                 Col-0-1_01frags Col-0-1_02frags Col-0-1_04frags Col-0-1_06frags Col-0-1_08frags \
#                 Col-0-2_01frags Col-0-2_02frags Col-0-2_04frags Col-0-2_06frags Col-0-2_08frags \
#                 Col-0-3_01frags Col-0-3_02frags Col-0-3_04frags Col-0-3_06frags Col-0-3_08frags \
#                 Col-0-4_01frags Col-0-4_02frags Col-0-4_04frags Col-0-4_06frags Col-0-4_08frags \
#                 Col-0-5_01frags Col-0-5_02frags Col-0-5_04frags Col-0-5_06frags Col-0-5_08frags \
#                 Col-0-6_01frags Col-0-6_02frags Col-0-6_04frags Col-0-6_06frags Col-0-6_08frags \
#                 Col-0-7_01frags Col-0-7_02frags Col-0-7_04frags Col-0-7_06frags Col-0-7_08frags \
#                 Col-0-8_01frags Col-0-8_02frags Col-0-8_04frags Col-0-8_06frags Col-0-8_08frags'.split()
# duplex_files = [f'data/variant/big_{s}_duplex_strandsup.tsv' for s in sample_names] # output of duplex_caller prefiltered to require coverage in both strands and 24% support in both strands
# informed_freq_files = [f'data/variant/big_{s}_informed_freq.tsv' for s in sample_names] # output of dup_informed_caller --min_support 2. Used to find the frequency of each mutations in the sample 
# informed_cont_files = [f'data/variant/big_{s}_informed_control.tsv' for s in sample_names[:8]] # output of dup_informed_caller --min_support 2 --duplicate_support 2. Used to filter out cryptic dups and common PCR errors

sample_names = ['chandler']
duplex_files = ['data/variant/chandler_KVKCS001D_0_filtered_duplex_strandsup.tsv']
informed_freq_files = ['data/variant/chandler_KVKCS001D_0_filtered_informed_freq.tsv']
informed_cont_files = [f'data/variant/big_{s}_informed_control.tsv' for s in 'Col-0-1 Col-0-2 Col-0-3 Col-0-4 Col-0-5 Col-0-6 Col-0-7 Col-0-8'.split()]

# coverage_files = [f'../data/coverage/total/big_{name}.bg' for name in sample_names[:8]]
ref_index = 'data/ref/ref.fa.fai'
blacklist_prefix = 'data/blacklist/arabidopsis/duplex_blacklist_'

output_files = [f'data/variant/big_{s}_mutations.tsv' for s in sample_names]


In [ ]:
df = pd.read_table(informed_freq_files[0], quoting=csv.QUOTE_NONE)

In [ ]:
df[df.pos == 8582750]

# Load blacklists

In [ ]:
chrom_sizes = gtools.load_chrom_sizes(ref_index)

In [ ]:
# all are indexed as bl[chrom][pos] = value
bl_cov_per = dict() # coverage percentile
bl_dup_dist = dict() # mismatches required to have a duplicate 120mer in the genome
bl_poly_at = dict() # length of poly-A or poly-T starting or ending at position
bl_poly_rep = dict() # length of di or trinucleotide repeat starting or ending at position
for chrom in tqdm(chrom_sizes):
    bl_cov_per[chrom] = np.load(f'{blacklist_prefix}{chrom}_coverage_percentile.npy')
    bl_dup_dist[chrom] = np.load(f'{blacklist_prefix}{chrom}_duplicate_distance.npy')
    bl_poly_at[chrom] = np.load(f'{blacklist_prefix}{chrom}_poly_at_length.npy')
    bl_poly_rep[chrom] = np.load(f'{blacklist_prefix}{chrom}_poly_repeat_length.npy')    

In [ ]:
np.count_nonzero((bl_cov_per['ChrC'] > 0.01))

# Add extra info needed for mutation calling to duplex_caller output

In [ ]:
def line_to_var(line):
    var = line[:-1].split('\t')
    var[1] = int(var[1]) # change position to an int
    return var

In [ ]:
# adds information used for filtering mutations
def add_mut_info(call_file, freq_file, control_files, output_file):
    # open all files
    f_call = open(call_file, 'r')
    f_freq = open(freq_file, 'r')
    f_conts = [open(f, 'r') for f in control_files]
    f_out = open(output_file, 'w')
    
    # extract the header from all files
    try:
        dcs = {x:i for i, x in enumerate(next(f_call)[:-1].split('\t'))} # duplex caller columns, key=column, value=column number
        ics = {x:i for i, x in enumerate(next(f_freq)[:-1].split('\t'))} # informed caller columns
        [next(f) for f in f_conts]
    except StopIteration:
        print(f'empty file in {call_file} or {freq_file}')
        return
    
    # write the output header with extra columns
    f_out.write('\t'.join(list(dcs.keys()) + 'high_sup avg_mq cov_per dup_dist poly_at poly_rep sample_sup sample_cov control_sup'.split()) + '\n')
    
    # load the first freq and control variants
    cur_freq = line_to_var(next(f_freq))
    cur_conts = [line_to_var(next(f)) for f in f_conts]
    
    # print(ics)
    # print(cur_freq)
    
    # for each variant in the call file, add extra information and output
    bar = tqdm(miniters=10000, maxinterval=100)
    while True:
        # get the next variant in the call file
        try:
            cur_var = line_to_var(next(f_call))
        except StopIteration:
            break
        
        # calculate the number of supporting reads with a high BQ value (>=30)
        if cur_var[dcs['ref']] == '*' or cur_var[dcs['alt']] == '*':
            high_sup = int(cur_var[dcs['f_sup']]) + int(cur_var[dcs['r_sup']])
        else:
            high_sup = sum([(ord(bq) - 33) >= 30 for bq in cur_var[dcs['bq']]])
        
        # calculate the average MQ of supporting reads
        avg_mq = sum([ord(mq) - 33 for mq in cur_var[dcs['mq']]]) / len(cur_var[dcs['mq']])
        
        # find the blacklist values
        cov_per = bl_cov_per[cur_var[dcs['chrom']]][int(cur_var[dcs['pos']])]
        dup_dist = bl_dup_dist[cur_var[dcs['chrom']]][int(cur_var[dcs['pos']])]
        poly_at = bl_poly_at[cur_var[dcs['chrom']]][int(cur_var[dcs['pos']])]
        poly_rep = bl_poly_rep[cur_var[dcs['chrom']]][int(cur_var[dcs['pos']])]
        
        # read in variants from the frequency and control files until at or past the current variant
        while cur_freq[:4] < cur_var[:4]:
            try:
                cur_freq = line_to_var(next(f_freq))
            except StopIteration:
                cur_freq = [chr(1000)]
            
        for i in range(len(f_conts)):
            while cur_conts[i][:4] < cur_var[:4]:
                try:
                    cur_conts[i] = line_to_var(next(f_conts[i]))
                except StopIteration:
                    cur_conts[i] = [chr(1000)]
        
        # add the support and coverage of the variant within the sample
        if cur_var[0] == 'Chr4' and cur_var[1] == 8582750:
            print(f'at var {cur_var}, cur_freq is {cur_freq}')
        if cur_freq[:4] == cur_var[:4]:
            sample_sup = int(cur_freq[ics['f_sup']]) + int(cur_freq[ics['r_sup']])
            sample_cov = int(cur_freq[ics['f_cov']]) + int(cur_freq[ics['r_cov']])
        else:
            sample_sup = 0
            sample_cov = 0
        
        # add the support of the variant across all control samples
        control_sup = 0
        for i in range(len(f_conts)):
            if cur_conts[i][:4] == cur_var[:4]:
                control_sup += int(cur_conts[i][ics['f_sup']]) + int(cur_conts[i][ics['r_sup']])
        
        cur_var += [high_sup, avg_mq, cov_per, dup_dist, poly_at, poly_rep, sample_sup, sample_cov, control_sup]
        
        if True:
            f_out.write('\t'.join(map(str, cur_var)) + '\n')
        
        bar.update()
            
    f_call.close()
    f_freq.close()
    for f in f_conts:
        f.close()
    f_out.close()

In [ ]:
if __name__ == '__main__':
    # start worker processes from pool
    pool = multiprocessing.Pool(processes=8)
    processes = []
    for i, sample in enumerate(sample_names):
        f_call = duplex_files[i]
        f_freq = informed_freq_files[i]
        f_conts = [f for f in informed_cont_files if sample[:7] not in f] # don't use a sample as its own control
        
        if len([x for x in informed_cont_files if x not in f_conts]) > 0:
            print(f'not using control(s) {[x for x in informed_cont_files if x not in f_conts]} for sample {sample}')
        # print(f_call, f_freq, f_conts)
        processes.append(pool.apply_async(add_mut_info, (f_call, f_freq, f_conts, f'tmp/added_info_{sample_names[i]}.tsv')))
        
    pool.close()
    pool.join()

    # check for errors
    for p in processes:
        p.get()

# Call mutations using each modified output

In [ ]:
# Returs a boolean list of whether each row of the df passes the contaminant filter
# Variants fail this filter if their fragment has 2 variants >max_dist bp apart
# max_dist=0 never allows more than one variant in a fragment, max_dist=1 allows a pair of back-to-back variants
# input df must be sorted by chrom, frag_start, frag_len, frag_umi, pos
def not_contaminant(df, max_dist):
    df.frag_umi = df.frag_umi.fillna('') # need to set NAs to something else, as NA != NA
    rows = df.itertuples()
    prev_frag = ('')
    keep = []
    poss = []
    while True:
        r = next(rows, None)
        if r is None or (r.chrom, r.frag_start, r.frag_len, r.frag_umi) != prev_frag:
            keep += [poss[-1] - poss[0] <= max_dist] * len(poss) if len(poss) > 0 else []
            if r is None:
                return keep
            prev_frag = (r.chrom, r.frag_start, r.frag_len, r.frag_umi)
            poss = []
        poss.append(r.pos)

In [ ]:
# returns a df with only variants which pass all these filter cutoffs
def filter_vars(df, min_strand_frac=0.76, min_strand_cov=2, min_total_cov=6, min_high_sup=4, min_avg_mq=10, max_emm=0.21, \
                min_cov_per=0.007, min_dup_dist=-1, max_poly_at=7, max_poly_rep=4, max_cont_dist=4, min_indel_end_dist=6, \
                max_control_sup=6, require_overlap=False, max_frag_len=2000): # FIXME return max_frag_len to 300
    
    # do most filters, but not require_overlap nor the full strand frac (just require 0.24 for now)
    df = df[(df.f_read1_sup / df.f_read1_cov >= 0.24) & (df.r_read1_sup / df.r_read1_cov >= 0.24) & \
            (df.f_read1_cov >= min_strand_cov) & (df.r_read1_cov >= min_strand_cov) & \
            (df.f_read1_cov + df.r_read1_cov >= min_total_cov) & \
            (df.high_sup >= min_high_sup) & \
            (df.avg_mq >= min_avg_mq) & \
            (df.end_mismatch_rate <= max_emm) & \
            (df.cov_per >= min_cov_per) & \
            # (df.cov_per <= 1 - min_cov_per) & \
            (df.dup_dist >= min_dup_dist) & \
            (df.poly_at <= max_poly_at) & (df.poly_rep <= max_poly_rep) & \
            (df.control_sup <= max_control_sup) & \
            (df.frag_len <= max_frag_len)]
    
    
    if len(df) == 0:
        return df
    
    keep = []
    for r in df.itertuples():
        if r.ref != '*' and r.alt != '*':
            k = True
        elif r.alt == '*' and r.pos > r.frag_start + r.frag_len // 2: # if a deletion nearer the end of the fragment
            k = r.frag_start + r.frag_len - r.pos - len(r.alt) >= min_indel_end_dist # count distance from the last base of the deletion
        else:
            k = r.pos - r.frag_start >= min_indel_end_dist and r.frag_start + r.frag_len - r.pos - 1 >= min_indel_end_dist
        keep.append(k)
    df = df[keep]
    
    df = df.sort_values('chrom frag_start frag_len frag_umi pos ref alt'.split())
    df = df[not_contaminant(df, max_cont_dist)]
    
    if len(df) == 0:
        return df
    
    # now that contaminants are removed, require overlap and full read support (contaminants are less likely to be filtered if these filters are run first)
    if require_overlap:
        df = df[(df.f_sup > 0) & (df.r_sup > 0)]
    df = df[(df.f_read1_sup / df.f_read1_cov >= min_strand_frac) & (df.r_read1_sup / df.r_read1_cov >= min_strand_frac)]
    
    return df

In [ ]:
from pandas.errors import EmptyDataError 

# load each set of variants with the added info, then filter to only mutations
dfs_mut = []
for i in range(len(sample_names)):
    try:
        df = pd.read_table(f'tmp/added_info_{sample_names[i]}.tsv', quoting=csv.QUOTE_NONE)
    except EmptyDataError:
        print(f'no mutations for sample {sample_names[i]}')
        dfs_mut.append(None)
        continue
    print(f'{sample_names[i]} loaded {len(df)} raw mutations')

    if 'swapped' in sample_names[i]: # for the swapped samples, it wouldn't be fair to include the sample's own reads in the control support
        df.control_sup -= 2
    
    if 'Ler-0-germline' in sample_names[i]:
        df_fil = filter_vars(df, max_cont_dist=1000)
    else:
        df_fil = filter_vars(df)
        
    print(f'{len(df_fil)} mutations remain after filters')
    
    df_fil = df_fil.drop_duplicates('chrom pos ref alt'.split())
    print(f'{len(df_fil)} unique mutations remain after filters')
    
    dfs_mut.append(df_fil)

In [ ]:
# label all the mutations which are present in a sibling plant
for i in tqdm(range(len(sample_names))):
    if dfs_mut[i] is None:
        continue
    sibling_muts = set()
    for j in range(len(sample_names)):
        if i == j or sample_names[i].rsplit('-', 1)[0] != sample_names[j].rsplit('-', 1)[0] or dfs_mut[j] is None:
            continue
        df = dfs_mut[j]
        sibling_muts.update(zip(df.chrom, df.pos, df.ref, df.alt))
    
    dfs_mut[i]['in_sibling'] = dfs_mut[i].apply(lambda r: (r.chrom, r.pos, r.ref, r.alt) in sibling_muts, axis=1)

In [ ]:
df_tmp = dfs_mut[0].copy()
df_tmp = df_tmp[(df_tmp.chrom == 'Chr4') & (df_tmp.pos > 8500000) & (df_tmp.pos < 10000000)]
df_tmp

# Remove contaminants (mutations shared between non-sibling samples)

In [ ]:
# for each pair of samples, get all the "contaminant" mutations found in both
df_contam = pd.DataFrame(columns=dfs_mut[0].columns)
for i in range(len(sample_names)):
    for j in range(len(sample_names)):
        if sample_names[i].rsplit('-', 1)[0] == sample_names[j].rsplit('-', 1)[0]: # ignore siblings
            continue
        if len(dfs_mut[i]) > 10000 or len(dfs_mut[j]) > 10000: # ignore samples with large numbers of mutations (the Ler-0 germline mutations)
            continue
        if 'swapped' in sample_names[i] or 'swapped' in sample_names[j]: # ignore swapped samples
            continue
        
        df = dfs_mut[i].merge(dfs_mut[j]['chrom pos alt ref'.split()], how='inner')
        if len(df) > 0:
            print(f'{sum(df.in_sibling)} germline and {sum(~df.in_sibling)} somatic mutations in {sample_names[i]} are in {sample_names[j]}')

        df_contam = df_contam.append(df)
df_contam = df_contam.sort_values('chrom pos ref alt'.split())
contaminants = set(zip(df_contam.chrom, df_contam.pos, df_contam.ref, df_contam.alt))
print(f'{len(contaminants)} unique and {len(df_contam)} total contaminant mutations')

In [ ]:
# remove all contaminant mutations
for i in range(len(dfs_mut)):
    if 'swapped' in sample_names[i]: # don't discard "contaminants" for the swapped samples
        continue
    
    dfs_mut[i] = dfs_mut[i][dfs_mut[i].apply(lambda r: (r.chrom, r.pos, r.ref, r.alt) not in contaminants, axis=1)]

In [ ]:
print('Final mutation counts (unique and in sibling):')
for i in range(len(dfs_mut)):
    print(f'{sample_names[i]}\t{sum(~dfs_mut[i].in_sibling)}\t{sum(dfs_mut[i].in_sibling)}')

In [ ]:
# save the final mutation set
for i in range(len(dfs_mut)):
    dfs_mut[i].to_csv(output_files[i], sep='\t', index=False)

In [ ]:
plt.hist(dfs_mut[0].frag_len)

In [ ]:
df_examples = dfs_mut[1][(dfs_mut[1].chrom == 'Chr5') & (np.abs(dfs_mut[1].pos - 24022141) < 300)]
df_examples = df_examples.append(dfs_mut[1][(dfs_mut[1].chrom == 'Chr1') & (np.abs(dfs_mut[1].pos - 10497750) < 300)])
df_examples = df_examples.append(dfs_mut[1][(dfs_mut[1].chrom == 'Chr5') & (np.abs(dfs_mut[1].pos - 17567566) < 300)])
df_examples = df_examples.append(dfs_mut[1][(dfs_mut[1].chrom == 'Chr1') & (np.abs(dfs_mut[1].pos - 16015120) < 300)])

In [ ]:
gtools.sam_view_df(df_examples, ['data/align/big_Col-0-2_merged.bam', 'data/align/big_Col-0-3_merged.bam'])

In [ ]:
df_examples

# Testing filter sets

## Check BQ and MQ histogram

In [ ]:
aln = pysam.AlignmentFile('data/align/chandler_KVKCS001D_0_filtered.bam')
bqs = np.zeros(50, dtype='i')
mqs = np.zeros(50, dtype='i')
i = 0
mismatches = 0
total = 0
for read in tqdm(aln.fetch()):
    i += 1
    if i > 50000:
        break
    for bq in read.get_forward_qualities():
        bqs[bq] += 1
    mqs[read.mapping_quality] += 1
    for x in read.get_aligned_pairs(with_seq=True):
        if x[2] is None or x[2].islower():
            mismatches += 1
        total += 1


In [ ]:
mismatches / total

In [ ]:
fig, axs = plt.subplots(2, sharex=True)
axs[0].bar(range(len(bqs)), bqs)
axs[1].bar(range(len(mqs)), mqs)
axs[0].set_title('BQs')
axs[1].set_title('MQs')

## Find optimal cutoff for each filter

In [ ]:
# real_samples = list(range(1, 5))
# fake_samples = list(range(1, 4))

real_samples = [0]
fake_samples = []

In [ ]:
# load Col-0-{1-4} as a training dataset. Also load the swapped samples for estimating FP rate
df_real = None
for i in tqdm(real_samples):
    df = pd.read_table(f'tmp/added_info_{sample_names[i]}.tsv', quoting=csv.QUOTE_NONE)
    print(f'Loaded {len(df)} raw mutations from {sample_names[i]}')
    
    df = df[df.control_sup < 100]
    print(f'{len(df)} after pre-filter')
    
    if df_real is None:
        df_real = df
    else:
        df_real = df_real.append(df)
df_real = df_real.sort_values('chrom frag_start frag_len frag_umi pos ref alt'.split()) # first by fragment (filter_vars will do this, but doing it here keeps the order stable)

In [ ]:
df_fake = pd.DataFrame(columns=df_real.columns)
for i in tqdm(fake_samples):
    df = pd.read_table(f'tmp/added_info_{sample_names[i]}.tsv', quoting=csv.QUOTE_NONE)
    print(f'Loaded {len(df)} raw mutations')
    
    df.control_sup -= 2 # for the swapped samples, it wouldn't be fair to include the sample's own reads in the control support
    
    df = df[df.control_sup < 100]
    print(f'{len(df)} after pre-filter')
    
    df_fake = df_fake.append(df)
df_fake = df_fake.sort_values('chrom frag_start frag_len frag_umi pos ref alt'.split()) # first by fragment (filter_vars will do this, but doing it here keeps the order stable)

In [ ]:
real_cov, fake_cov = 1, 1 # this can be changed back to zero, not that it makes a difference
for i in real_samples:
    for chrom in chrom_sizes:
        real_cov += np.sum(np.load(f'data/coverage/2f1r/big_{sample_names[i]}_{chrom}.npy'))
for i in fake_samples:
    for chrom in chrom_sizes:
        fake_cov += np.sum(np.load(f'data/coverage/2f1r/big_{sample_names[i]}_{chrom}.npy'))
real_cov, fake_cov

In [ ]:
# returns the fraction of variants which are C->T
def ct_rate(df):
    # df = df[(df.ref != '*') & (df.alt != '*')]
    
    if len(df) == 0:
        return np.nan
    return sum(((df.ref == "C") & (df.alt == "T")) | ((df.ref == "G") & (df.alt == "A"))) / len(df)

# returns the fraction of variants which are indels
def indel_rate(df):
    if len(df) == 0:
        return np.nan
    return sum((df.ref == "*") | (df.alt == "*")) / len(df)


In [ ]:
# max_ct = 0.49 # ct_rate(filter_vars(df_real, min_strand_cov=4, min_strand_frac=0.99, max_emm=0.3)) # 0.494 is the C>T rate of the weng et al unique mutations
# min_ct = ct_rate(df_real)
# min_indel = indel_rate(filter_vars(df_real, min_strand_cov=4, max_emm=0.3))
# min_ct, max_ct

In [ ]:
def fp_rates(df_real_fil, df_fake_fil):
    swapped_fp_rate_est = (len(df_fake_fil) / fake_cov) / (len(df_real_fil) / real_cov) if len(df_real_fil) > 0 else np.nan
    swapped_tp_count_est = max(0, len(df_real_fil) * (1 - swapped_fp_rate_est))
    
    return swapped_fp_rate_est, swapped_tp_count_est

## Compare the output of two filter sets

In [ ]:
# use one set of filters here
df_real_fil_1 = filter_vars(df_real, min_high_sup=1)
df_fake_fil_1 = filter_vars(df_fake, min_high_sup=1)
len(df_real_fil_1), len(df_fake_fil_1), fp_rates(df_real_fil_1, df_fake_fil_1)

In [ ]:
# and a different set here (the output will be more useful if this one calls more mutations)
df_real_fil_2 = filter_vars(df_real, min_high_sup=0)
df_fake_fil_2 = filter_vars(df_fake, min_high_sup=0)
len(df_real_fil_2), len(df_fake_fil_2), fp_rates(df_real_fil_2, df_fake_fil_2)

In [ ]:
# get the variants called with one filter set but not the other
df_unique = df_real_fil_2.merge(df_real_fil_1['chrom pos alt ref'.split()], on='chrom pos alt ref'.split(), how='outer', indicator=True)
df_unique = df_unique[df_unique._merge != 'both']

df_unique

In [ ]:
# df_tmp = df_unique.sample(10)
# gtools.sam_view_df(df_tmp, [f'data/align/big_Col-0-{i}_merged.bam' for i in range(1, 5)], max_reads=50000, window_size=1)
# df_tmp

# See what happens as filters are adjusted

In [ ]:
2 - np.nan

In [ ]:
param_to_desc = {
    'min_strand_cov': 'min read coverage in each strand',
    'min_total_cov': 'min total read coverage',
    'min_high_sup': 'min support from high BQ (>30) reads',
    'min_avg_mq': 'min average MQ of supporting reads',
    'max_emm': 'max mismatch rate before an end is trimmed',
    'min_indel_end_dist': 'min distance from an indel to fragment end',
    'min_cov_per': 'min coverage percentile of genomic site',
    'min_dup_dist': "min hamming distance to another fragment in the genome",
    'max_poly_at': 'max length of poly-A/T repeat',
    'max_poly_rep': 'max length of di/trinucleotide repeat',
    'max_control_sup': 'max (non-sequencing error) occurences in control samples',
    'max_frag_len': 'max fragment length',
    'max_cont_dist': 'max distance between any 2 variants in a fragment',
    'min_strand_frac': 'min fraction of reads in support from each strand',
    'require_overlap': 'require (1) or don\'t require (0) read1-read2 overlap',
}

cur_cutoffs = {
    'min_strand_cov':2,
    'min_total_cov':6,
    'min_high_sup':4,
    'min_avg_mq':10,
    'max_emm':0.21,
    'min_indel_end_dist':6,
    'min_cov_per':0.007,
    'min_dup_dist':0,
    'max_poly_at':7,
    'max_poly_rep':4,
    'max_control_sup':6,
    'max_frag_len':300,
    'max_cont_dist':4,
    'min_strand_frac':0.76,
    'require_overlap':0,
}

sliders = [
    [{'min_strand_cov':x} for x in range(10)],
    [{'min_total_cov':x} for x in range(20)],
    [{'min_high_sup':x} for x in range(20)],
    [{'min_avg_mq':x} for x in range(44)],
    [{'max_emm':x / 40} for x in range(42)],
    [{'min_indel_end_dist':x} for x in range(20)],
    [{'min_cov_per':x / 1000} for x in range(52)],
    [{'min_dup_dist':x} for x in range(5)],
    [{'max_poly_at':x} for x in range(10)],
    [{'max_poly_rep':x} for x in range(10)],
    [{'max_control_sup':x} for x in range(50)],
    [{'max_frag_len':x} for x in range(250, 350, 10)],
    [{'max_cont_dist':x} for x in range(10)],
    [{'min_strand_frac':x / 20} for x in range(22)],
    [{'require_overlap':x} for x in [False, True]],
]

fig, axs = plt.subplots(math.ceil(len(sliders) / 3), 3, figsize=(20, 12), gridspec_kw={'hspace':0.33}, sharey=True)
for i, slider in tqdm(enumerate(sliders)):
    # if i != 4:
    #     continue
    swap_fpr = []
    swap_tpc = []
    ct_rates = []
    for filter_set in slider:
        df_r = filter_vars(df_real, **filter_set) # .drop_duplicates('chrom pos ref alt'.split())
        df_f = filter_vars(df_fake, **filter_set)
        rates = fp_rates(df_r, df_f)
        swap_fpr.append(min(0.2, rates[0]) if not np.isnan(rates[0]) else np.nan) # output a max FP rate of 0.2, keep nans (min(x, nan) evalues to x)
        swap_tpc.append(rates[1])
        ct_rates.append(ct_rate(df_r))
    ct_max = max([x for x in ct_rates if not np.isnan(x)]) # calculate the max ct within the subplot
    ct_depletion = [min(0.2, ct_max - x) if not np.isnan(x) else np.nan for x in ct_rates] # calculate the max - threshold ct rate, limit to 0.2 and propagate nans
    
    row = i // 3
    col = i % 3
    
    x_axis = list(slider[0].keys())[0]
    x_values = [filter_set[x_axis] for filter_set in slider]
    ax_count = axs[row, col].twinx()
    a, = axs[row, col].plot(x_values, swap_fpr, c=plt.get_cmap('tab20')(6))
    b, = axs[row, col].plot(x_values, ct_depletion, c=plt.get_cmap('tab20')(7))
    c, = ax_count.plot(x_values, swap_tpc, c=plt.get_cmap('tab20')(0))
    axs[row, col].set_xlabel(param_to_desc[x_axis])
    d = axs[row, col].vlines(cur_cutoffs[x_axis], ymin=-1, ymax=1, color='black')
    
    
axs[0, 0].set_ylim(0, 0.205)
fig.legend([a, b, c, d], ['FP rate (est swapped samples)', 'C>T rate below max (max rate within plot - rate at threshold)', 'TPs detected (est swapped samples)', 'selected cutoff'], loc='upper center')
fig.savefig('figs/varying_nanoseq_mutation_filtering_cutoffs.png', dpi=300, bbox_inches='tight')

## Check for increased mutation rate near fragment ends

In [ ]:
bin_size = 3
max_end_dist = 75

df_seq_err = None
df_pcr_err = None
frag_end_cov = np.zeros(150)
i = 0
for df_chunk in tqdm(pd.read_table('data/variant/big_Col-0-1_duplex.tsv', quoting=csv.QUOTE_NONE, chunksize=100000)):
    df_s = df_chunk[((df_chunk.f_sup == 1) & (df_chunk.r_sup == 0) & (df_chunk.r_cov > 0)) | ((df_chunk.r_sup == 1) & (df_chunk.f_sup == 0) & (df_chunk.f_cov > 0))]
    df_p = df_chunk[((df_chunk.f_read1_sup == df_chunk.f_read1_cov) & (df_chunk.r_read1_sup == 0) & (df_chunk.r_read1_cov > 0)) | 
                    ((df_chunk.r_read1_sup == df_chunk.r_read1_cov) & (df_chunk.f_read1_sup == 0) & (df_chunk.f_read1_cov > 0))]
    
    df_seq_err = df_s if df_seq_err is None else df_seq_err.append(df_s)
    df_pcr_err = df_p if df_pcr_err is None else df_pcr_err.append(df_p)
    
    for frag_len in df_chunk.frag_len:
        frag_end_cov[:min(150, frag_len // 2)] += 2
    if i > 100:
        break
    i += 1

# bin the fragment coverage
frag_cov_binned = np.ones(max_end_dist // bin_size)
for i in range(len(frag_end_cov)):
    if i // bin_size < len(frag_cov_binned):
        frag_cov_binned[i // bin_size] += frag_end_cov[i]

In [ ]:
def snps_and_indels_over_frag(df):
    frag_indels = np.zeros(max_end_dist // bin_size)
    frag_snvs = np.zeros(max_end_dist // bin_size)
    for r in df.itertuples():
        p = min(r.pos - r.frag_start, r.frag_start + r.frag_len - r.pos) // bin_size
        if p >= len(frag_snvs):
            continue
        if r.ref == '*' or r.alt == '*':
            frag_indels[p] += 1
        else:
            frag_snvs[p] += 1

    return frag_snvs, frag_indels

In [ ]:
def bounds(arr):
    intervals = np.zeros((2, len(arr)))
    for i in range(len(arr)):
        intervals[:, i] = stats.binom.interval(0.95, sum(arr), arr[i] / sum(arr))
    return intervals

In [ ]:
# called_snp, called_indel = snps_and_indels_over_frag(filter_vars(df_real))
# unfiltered_snp, unfiltered_indel = snps_and_indels_over_frag(df_real.sample(100000))
# seq_err_snp, seq_err_indel = snps_and_indels_over_frag(df_seq_err)
# pcr_err_snp, pcr_err_indel = snps_and_indels_over_frag(df_pcr_err)

fig, axs = plt.subplots(2, figsize=(6, 3), sharex=True)
x = [x * bin_size for x in range(len(called_snp))]

axs[0].set_title('SNVs', y=0.7)
axs[0].plot(x, called_snp / sum(called_snp) / frag_cov_binned)
axs[0].plot(x, unfiltered_snp / sum(unfiltered_snp) / frag_cov_binned)
axs[0].plot(x, seq_err_snp / sum(seq_err_snp) / frag_cov_binned)
axs[0].plot(x, pcr_err_snp / sum(pcr_err_snp) / frag_cov_binned)
axs[0].legend(['called mutations', 'unfiltered variants', 'sequencing errors', 'PCR errors'], bbox_to_anchor=(1,1), loc='upper left')

axs[0].fill_between(x, bounds(called_snp)[0] / sum(called_snp) / frag_cov_binned, bounds(called_snp)[1] / sum(called_snp) / frag_cov_binned, alpha=0.2)
axs[0].fill_between(x, bounds(unfiltered_snp)[0] / sum(unfiltered_snp) / frag_cov_binned, bounds(unfiltered_snp)[1] / sum(unfiltered_snp) / frag_cov_binned, alpha=0.2)
axs[0].fill_between(x, bounds(seq_err_snp)[0] / sum(seq_err_snp) / frag_cov_binned, bounds(seq_err_snp)[1] / sum(seq_err_snp) / frag_cov_binned, alpha=0.2)
axs[0].fill_between(x, bounds(pcr_err_snp)[0] / sum(pcr_err_snp) / frag_cov_binned, bounds(pcr_err_snp)[1] / sum(pcr_err_snp) / frag_cov_binned, alpha=0.2)

axs[1].set_title('indels', y=0.7)
axs[1].plot(x, called_indel / sum(called_indel) / frag_cov_binned)
axs[1].plot(x, unfiltered_indel / sum(unfiltered_indel) / frag_cov_binned)
axs[1].plot(x, seq_err_indel / sum(seq_err_indel) / frag_cov_binned)
axs[1].plot(x, pcr_err_indel / sum(pcr_err_indel) / frag_cov_binned)

axs[1].fill_between(x, bounds(called_indel)[0] / sum(called_indel) / frag_cov_binned, bounds(called_indel)[1] / sum(called_indel) / frag_cov_binned, alpha=0.2)
axs[1].fill_between(x, bounds(unfiltered_indel)[0] / sum(unfiltered_indel) / frag_cov_binned, bounds(unfiltered_indel)[1] / sum(unfiltered_indel) / frag_cov_binned, alpha=0.2)
axs[1].fill_between(x, bounds(seq_err_indel)[0] / sum(seq_err_indel) / frag_cov_binned, bounds(seq_err_indel)[1] / sum(seq_err_indel) / frag_cov_binned, alpha=0.2)
axs[1].fill_between(x, bounds(pcr_err_indel)[0] / sum(pcr_err_indel) / frag_cov_binned, bounds(pcr_err_indel)[1] / sum(pcr_err_indel) / frag_cov_binned, alpha=0.2)

axs[1].set_xlabel('Distance from fragment end (bp)')
axs[0].set_ylabel('Mutation/error rate', ha='right')
fig.savefig('figs/mutation_rate_across_fragment_length.svg', dpi=300, bbox_inches='tight')

## Use subsampled files to determine min required coverage

In [ ]:
subsample_levels = [(1, 1, 3), (1, 2, 3), (2, 2, 3), (3, 3, 3)] # (f coverage, r coverage, replicates)
sub_files = [] # subsample files generated by duplex_subsampler
for x in subsample_levels:
    for i in range(x[2]):
        sub_files.append(f'../data/variant/subsample/big_Col-0-1_{x[0]}f{x[1]}rpairs_{i}_duplex_strandsup.tsv')
high_file = '../data/variant/subsample/big_Col-0-1_ge4pairs_duplex_strandsup.tsv'

In [ ]:
if __name__ == '__main__':
    # start worker processes from pool
    with multiprocessing.Pool(processes=8) as pool:
        processes = []
        naive_controls = naive_var_files[1:8]
        informed_controls = informed_var_files[1:8]
        for i in range(len(sub_files)):
            processes.append(pool.apply_async(add_mut_info, (sub_files[i], naive_controls, informed_controls, f'tmp/added_info_{i}.tsv')))
        processes.append(pool.apply_async(add_mut_info, (high_file, naive_controls, informed_controls, f'tmp/added_info_{len(sub_files)}.tsv')))
        pool.close()
        pool.join()

    # check for errors
    for p in processes:
        p.get()

In [ ]:
df

In [ ]:
# load each set of variants with the added info, then filter to only mutations
dfs_sub = []
for i in tqdm(range(len(sub_files) + 1)):
    df = pd.read_table(f'tmp/added_info_{i}.tsv', quoting=csv.QUOTE_NONE)
    print(f'Loaded {len(df)} raw mutations')
    
    df['f_read1_frac'] = df.f_read1_sup / df.f_read1_cov
    df['r_read1_frac'] = df.r_read1_sup / df.r_read1_cov
    df['f_frac'] = df.f_sup / df.f_cov
    df['r_frac'] = df.r_sup / df.r_cov
    df.worst_frac = df.worst_frac.fillna(0)
    df.total_frac = df.total_frac.fillna(0)
    
    df = filter_vars(df, 0.76, -1, 2, -1, 4, 10, 0.21, 0.01, -1, 5, 3, 1, 1, 1, 8, 10)
    df = df.drop_duplicates('chrom pos ref alt'.split()) # keep only one copy of a mutation
    print(f'{len(df)} unique mutations remain after filters')
    
    dfs_sub.append(df)

df_high = dfs_sub[-1]
dfs_sub = dfs_sub[:-1]

In [ ]:
# TODO check how many mutations don't have enough support in the high coverage file, but do in the subsampled files (FP)
for i in range(len(dfs_sub)):
    dfs_sub[i] = dfs_sub[i].merge(df_high['chrom pos ref alt'.split()], how='outer', indicator='found')

In [ ]:
s = []
for sub in subsample_levels:
    s += [f'{sub[0]}f{sub[1]}r'] * sub[2]
for i in range(len(dfs_sub)):
    tp = sum(dfs_sub[i].found == 'both')
    fp = sum(dfs_sub[i].found == 'left_only')
    fn = sum(dfs_sub[i].found == 'right_only')
    print(f'{s[i]}\tTP: {tp}\tFP: {fp}\tFN: {fn}')

In [ ]:
df_tmp = dfs_sub[0].append(dfs_sub[1]).append(dfs_sub[2])#[dfs_sub[0].found == 'left_only']
gtools.sam_view_df(df_tmp, ['../data/align/big_Col-0-1_merged.bam', '../data/align/subsample/big_Col-0-1_1f1rpairs_0.bam', '../data/align/subsample/big_Col-0-1_1f1rpairs_1.bam', '../data/align/subsample/big_Col-0-1_1f1rpairs_2.bam', '../data/align/subsample/big_Col-0-1_ge4pairs.bam'], run=False)
df_tmp

# Misc